<a href="https://colab.research.google.com/github/monicafar147/regression-predict-api-template/blob/Monique1/Zindi_challenge_ct_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   make sure you are working in your own branch that you cloned from git
*   to commit your notebook changes to git:
1.   file
2.   save a copy in github
3.   choose your branch from branches dropdown
4.   change commit message to what you did e.g. "fixed dataframe indexes"
5.   click ok





#Data Preprocessing Tools

##Importing the libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##Importing the dataset

In [0]:
dataset = pd.read_csv('Train.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [0]:
print(X)

[['Order_No_4211' 'User_Id_633' 'Bike' ... -1.3004062 36.829741
  'Rider_Id_432']
 ['Order_No_25375' 'User_Id_2285' 'Bike' ... -1.2950041
  36.814358299999995 'Rider_Id_856']
 ['Order_No_1899' 'User_Id_265' 'Bike' ... -1.300921 36.828195
  'Rider_Id_155']
 ...
 ['Order_No_2831' 'User_Id_2956' 'Bike' ... -1.2584143 36.8048002
  'Rider_Id_697']
 ['Order_No_6174' 'User_Id_2524' 'Bike' ... -1.2792094999999999
  36.7948725 'Rider_Id_347']
 ['Order_No_9836' 'User_Id_718' 'Bike' ... -1.320157 36.830887
  'Rider_Id_177']]


In [0]:
print(y)

[ 745 1993  455 ... 2953 1380 2128]


## Taking care of missing data

## Encoding categorical data

### Encoding the Independent Variable

### Encoding the Dependent Variable

## Feature Scaling

#Dunyiswa

#Monica

#Monique

## Loading datasets

In [0]:
df_train = pd.read_csv('Train.csv')
df_riders = pd.read_csv('Riders.csv')

In [3]:
df_train.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3.0,Business,9.0,5.0,9:35:46 AM,9.0,5.0,9:40:10 AM,9.0,5.0,10:04:47 AM,9.0,5.0,10:27:30 AM,9.0,5.0,10:39:55 AM,4.0,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0
1,Order_No_25375,User_Id_2285,Bike,3.0,Personal,12.0,5.0,11:16:16 AM,12.0,5.0,11:23:21 AM,12.0,5.0,11:40:22 AM,12.0,5.0,11:44:09 AM,12.0,5.0,12:17:22 PM,16.0,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0
2,Order_No_1899,User_Id_265,Bike,3.0,Business,30.0,2.0,12:39:25 PM,30.0,2.0,12:42:44 PM,30.0,2.0,12:49:34 PM,30.0,2.0,12:53:03 PM,30.0,2.0,1:00:38 PM,3.0,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0
3,Order_No_9336,User_Id_1402,Bike,3.0,Business,15.0,5.0,9:25:34 AM,15.0,5.0,9:26:05 AM,15.0,5.0,9:37:56 AM,15.0,5.0,9:43:06 AM,15.0,5.0,10:05:27 AM,9.0,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341.0
4,Order_No_27883,User_Id_1737,Bike,1.0,Personal,13.0,1.0,9:55:18 AM,13.0,1.0,9:56:18 AM,13.0,1.0,10:03:53 AM,13.0,1.0,10:05:23 AM,13.0,1.0,10:25:37 AM,9.0,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214.0


In [4]:
df_riders.head()

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


## Summary statistics

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16291 entries, 0 to 16290
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   16291 non-null  object 
 1   User Id                                    16290 non-null  object 
 2   Vehicle Type                               16290 non-null  object 
 3   Platform Type                              16290 non-null  float64
 4   Personal or Business                       16290 non-null  object 
 5   Placement - Day of Month                   16290 non-null  float64
 6   Placement - Weekday (Mo = 1)               16290 non-null  float64
 7   Placement - Time                           16290 non-null  object 
 8   Confirmation - Day of Month                16290 non-null  float64
 9   Confirmation - Weekday (Mo = 1)            16290 non-null  float64
 10  Confirmation - Time   

At this stage, we notice that there are missing values from `Temperature` and `Precipitation in millimeters`. We also need to convert all the columns that contain time information from objects to a datetime format.

## Dealing with missing values

## Dealing with missing values

As described above, there are null-values present in the `Temperature` and `Precipitation in millimeters` columns.

We would be able to fill the missing `Temperature` values with values from the same dates where measurements were taken, but we don't know the actual date of each measurement. The easiest option would be to fill the missing values with the average of the column, but we should find a better way of doing this.

We are assuming that if `Precipitation in millimeters` is missing, that it was not raining on that day. We can therefore fill null values with 0.

In [0]:
# Fill missing precipitation values with 0
df_train['Precipitation in millimeters'] = df_train['Precipitation in millimeters'].fillna(value=0)

# Fill missing temperature values with averages
df_train['Temperature'] = df_train['Temperature'].fillna(df_train['Temperature'].mean())

## Fixing data types

The various columns that represent a time of day need to be converted to a datetime format.

In [0]:
df_train['Placement - Time'] = pd.to_datetime(df_train['Placement - Time']).dt.time
df_train['Confirmation - Time'] = pd.to_datetime(df_train['Confirmation - Time']).dt.time
df_train['Arrival at Pickup - Time'] = pd.to_datetime(df_train['Arrival at Pickup - Time']).dt.time
df_train['Pickup - Time'] = pd.to_datetime(df_train['Pickup - Time']).dt.time
df_train['Arrival at Destination - Time'] = pd.to_datetime(df_train['Arrival at Destination - Time']).dt.time

## Dropping columns

In [8]:
df_train['Vehicle Type'].unique()

array(['Bike', nan], dtype=object)

The `Vehicle Type` column only contains one value - Bike. This is redundant info, therefore we can drop this column.

In [0]:
df_train.drop(['Vehicle Type'], axis=1, inplace = True)

## Merge Train.csv with Riders.csv

In [10]:
merged_df = df_train.merge(df_riders, on='Rider Id')
merged_df.head()

,Order No,User Id,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_4211,User_Id_633,3.0,Business,9.0,5.0,09:35:46,9.0,5.0,09:40:10,9.0,5.0,10:04:47,9.0,5.0,10:27:30,9.0,5.0,10:39:55,4.0,20.4,0.0,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0,1637,1309,13.8,549
1,Order_No_27440,User_Id_2642,3.0,Personal,18.0,5.0,15:41:17,18.0,5.0,15:41:30,18.0,5.0,16:07:16,18.0,5.0,16:13:37,18.0,5.0,17:01:43,20.0,24.5,0.0,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886.0,1637,1309,13.8,549
2,Order_No_14170,User_Id_3339,3.0,Business,31.0,5.0,12:51:41,31.0,5.0,13:12:49,31.0,5.0,13:20:02,31.0,5.0,13:33:08,31.0,5.0,14:16:43,6.0,24.7,0.0,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615.0,1637,1309,13.8,549
3,Order_No_21727,User_Id_3523,3.0,Personal,2.0,2.0,07:12:10,2.0,2.0,07:12:29,2.0,2.0,07:35:24,2.0,2.0,07:38:46,2.0,2.0,08:28:32,18.0,15.2,0.0,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986.0,1637,1309,13.8,549
4,Order_No_8301,User_Id_87,2.0,Personal,22.0,2.0,10:40:58,22.0,2.0,10:42:24,22.0,2.0,10:56:00,22.0,2.0,10:59:40,22.0,2.0,11:26:22,7.0,19.2,0.0,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602.0,1637,1309,13.8,549


## Encoding data

In [0]:
# Reorder columns so that dependent variable is the last column of the dataframe
column_titles = [col for col in merged_df.columns if col!= 'Time from Pickup to Arrival'] + ['Time from Pickup to Arrival']
merged_df  = merged_df.reindex(columns=column_titles)

In [0]:
X = merged_df.iloc[:, :-1]
y = merged_df.iloc[:, -1]

#Rachel

#Wright